In [ ]:
!pip install ffmpeg-python
!pip install git+https://github.com/openai/whisper.git
!pip install jiwer
!pip install SpeechRecognition googletrans==4.0.0-rc1
!pip install TTS
!pip install colabcode
!pip install fastapi

In [ ]:
!pip install colabcode
!pip install fastapi

In [ ]:
#@title <h1>Step1: Setup Wav2Lip</h1>
# @markdown Magic script for wav2lip which:
# @markdown   * Installs dependency
# @markdown   * Downloads pretrained model
from IPython.display import HTML, clear_output
!rm -rf /content/sample_data
!mkdir /content/sample_data
S
!git clone https://github.com/justinjohn0306/Wav2Lip

%cd /content/Wav2Lip

#download the pretrained model
!wget 'https://github.com/justinjohn0306/Wav2Lip/releases/download/models/wav2lip.pth' -O 'checkpoints/wav2lip.pth'
!wget 'https://github.com/justinjohn0306/Wav2Lip/releases/download/models/wav2lip_gan.pth' -O 'checkpoints/wav2lip_gan.pth'
!wget 'https://github.com/justinjohn0306/Wav2Lip/releases/download/models/resnet50.pth' -O 'checkpoints/resnet50.pth'
!wget 'https://github.com/justinjohn0306/Wav2Lip/releases/download/models/mobilenet.pth' -O 'checkpoints/mobilenet.pth'
a = !pip install https://raw.githubusercontent.com/AwaleSajil/ghc/master/ghc-1.0-py3-none-any.whl
!pip install git+https://github.com/elliottzheng/batch-face.git@master

!pip install ffmpeg-python mediapipe==0.8.11

#this code for recording audio
"""
To write this piece of code I took inspiration/code from a lot of places.
It was late night, so I'm not sure how much I created or just copied o.O
Here are some of the possible references:
https://blog.addpipe.com/recording-audio-in-the-browser-using-pure-html5-and-minimal-javascript/
https://stackoverflow.com/a/18650249
https://hacks.mozilla.org/2014/06/easy-audio-capture-with-the-mediarecorder-api/
https://air.ghost.io/recording-to-an-audio-file-using-html5-and-js/
https://stackoverflow.com/a/49019356
"""
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data);
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});

</script>
"""

%cd /
from ghc.l_ghc_cf import l_ghc_cf
%cd content

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])

  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)

  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr


from IPython.display import HTML
from base64 import b64encode
def showVideo(path):
  mp4 = open(str(path),'rb').read()
  data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
  return HTML("""
  <video width=700 controls>
        <source src="%s" type="video/mp4">
  </video>
  """ % data_url)

from IPython.display import clear_output

clear_output()
print("All set and ready!")

All set and ready!


In [ ]:
import ffmpeg

input_file = 'spanish.mp4'
output_file = 'spanish.wav'

stream = ffmpeg.input(input_file)
stream = ffmpeg.output(stream, output_file, ab='160k', ac=2, ar=44100, vn=None)

try:
    ffmpeg.run(stream, capture_stderr=True)  # Capture stderr output in case of errors
    print("Conversion complete!")
except ffmpeg.Error as e:
    print(e.stderr.decode())  # Print the stderr output for debugging

Conversion complete!


In [ ]:
import whisper

model = whisper.load_model("small")
result = model.transcribe(audio="spanish.wav", language="Spanish")
print(result["text"])

 Mi nombre es Pedro Santa Cruz y este es el mejor proyecto que he visto en mi carrera. Exijo que este proyecto obtenga 100 automáticamente y les agradezco por darme el honor de enseñar a estudiantes tan maravillosos.


In [ ]:
%%time

from googletrans import Translator
translator = Translator()
translated = translator.translate(result["text"], dest='zh-cn')
print(translated.text)

我叫Pedro Santa Cruz，这是我职业生涯中见过的最好的项目。我要求该项目自动获得100个，我感谢您为我教授如此出色的学生的荣幸。
CPU times: user 151 ms, sys: 13 ms, total: 164 ms
Wall time: 794 ms


In [ ]:
%%time

import uuid
print(type(uuid.uuid4()))

<class 'uuid.UUID'>
CPU times: user 1.18 ms, sys: 21 µs, total: 1.2 ms
Wall time: 3.85 ms


In [ ]:
import torch
from TTS.api import TTS

# Get device
device = "cuda" if torch.cuda.is_available() else "cpu"

tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2").to(device)


 > tts_models/multilingual/multi-dataset/xtts_v2 is already downloaded.
 > Using model: xtts


In [ ]:
tts.tts_to_file(text=translated.text, language="zh-cn", speaker_wav="spanish.wav", file_path="english_translation.wav")


 > Text splitted to sentences.
['我叫Pedro Santa Cruz，这是我职业生涯中见过的最好的项目。', '我要求该项目自动获得100个，我感谢您为我教授如此出色的学生的荣幸。']
 > Processing time: 10.24227786064148
 > Real-time factor: 0.7242246883887399


'english_translation.wav'

In [ ]:
%cd /content/Wav2Lip
!python3 inference.py --checkpoint_path checkpoints/wav2lip.pth --face "../spanish.mp4" --audio "../english_translation.wav" --resize_factor 2


In [ ]:
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import FileResponse
import uvicorn
import pickle

app = FastAPI()

@app.on_event("startup")
def load_model():
    global model
    model = pickle.load(open("model_tree.pkl", "rb"))

@app.get('/')
def index():
    return {'message': 'This is the homepage of the API '}

@app.post("/upload_video/")
async def upload_video(video: UploadFile = File(...)):
    # Save the uploaded video to a temporary location
    with open("temp_video.mp4", "wb") as buffer:
        buffer.write(await video.read())

    # Return the video file as a response
    return FileResponse("temp_video.mp4", media_type="video/mp4")


ModuleNotFoundError: No module named 'fastapi'